In [44]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [45]:
total_dict = {}
county_dict = {}
state_dict = {}
dph_illinois_dict = {}
who_world_dict = {}
vul_dict = {}

In [46]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}
dynamic_illinois_dict = {}

1. Read County Data

In [47]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [48]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",398.0,1.0,3.0,0.0,2020-01-21,2020-07-04,day,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",459.0,22.0,5.0,0.0,2020-01-21,2020-07-04,day,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2110.0,31.0,206.0,0.0,2020-01-21,2020-07-04,day,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,None,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",10.0,4.0,0.0,0.0,2020-01-21,2020-07-04,day,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,None,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",66.0,1.0,0.0,0.0,2020-01-21,2020-07-04,day,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [49]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
# death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
# death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [50]:
log_case = case.apply(lambda x: np.log(x + 1))
log_death = death.apply(lambda x: np.log(x + 1))
log_case_per_100k_capita = case_per_100k_capita.apply(lambda x: np.log(x + 1))
log_death_per_100k_capita = death.apply(lambda x: np.log(x + 1))

1.1 Get Constant Classes from today's data

In [51]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

In [52]:
county_dict['case'] = {}
county_dict['death'] = {}
county_dict['case_per_100k_capita'] = {}
county_dict['death_per_100k_capita'] = {}

In [53]:
county_dict['case']['nolog'] = create_dict(case)
county_dict['case']['log'] = create_dict(log_case)
county_dict['death']['nolog'] = create_dict(death)
county_dict['death']['log'] = create_dict(log_death)
county_dict['case_per_100k_capita']['nolog'] = create_dict(case_per_100k_capita)
county_dict['case_per_100k_capita']['log'] = create_dict(log_case_per_100k_capita)
county_dict['death_per_100k_capita']['nolog'] = create_dict(death_per_100k_capita)
county_dict['death_per_100k_capita']['log'] = create_dict(log_death_per_100k_capita)


C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

In [54]:
county_dict

{'case': {'nolog': {'Quantiles': {'bins': '7.0,30.0,78.0,204.0,646.0000000000023,221395.0',
    'k': 6},
   'FisherJenks': {'bins': '2818.0,10447.0,26565.0,59829.0,107667.0,221395.0',
    'k': 6},
   'NaturalBreaks': {'bins': '2818.0,10447.0,26565.0,59829.0,107667.0,221395.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '2.0794415416798357,3.4339872044851463,4.3694478524670215,5.3230099791384085,6.4723462945009045,12.307708232246132',
    'k': 6},
   'FisherJenks': {'bins': '1.0986122886681098,2.995732273553991,4.430816798843313,5.846438775057725,7.589841512182657,12.307708232246132',
    'k': 6},
   'NaturalBreaks': {'bins': '1.0986122886681098,2.9444389791664403,4.356708826689592,5.7745515455444085,7.539027055823995,12.307708232246132',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,1.0,4.0,19.0,22630.0', 'k': 6},
   'FisherJenks': {'bins': '220.0,748.0,1619.0,2730.0,4622.0,22630.0', 'k': 6},
   'NaturalBreaks': {'bins': '220.0,748.0,1558.0,2730.0,4622.0,22630.0',
 

1.2 Create Dynamic classes

In [55]:
# case_ts = counties_gdf['cases_ts'].apply(lambda x: x.split(','))
# death_ts = counties_gdf['deaths_ts'].apply(lambda x: x.split(','))

In [56]:
# length = len(case_ts[1])

In [57]:
# new_case_dict = {}
# for i in range(1,length):
#     q6 = mc.Quantiles(case_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

In [58]:
# new_death_dict = {}
# for i in range(1,length):
#     d6 = mc.Quantiles(death_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
#     new_death_dict[i] = ','.join(d6.bins.astype(int).astype(str))

In [59]:
# dynamic_county_dict['new_case'] = new_case_dict
# dynamic_county_dict['new_death'] = new_death_dict


2.Read State Data

In [60]:
state_gdf = gpd.read_file('nyt_states_data.geojson')

In [61]:
state_gdf.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3205.0,79.0,94.0,1.0,2020-01-21,2020-07-04,day,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",190044.0,11458.0,3701.0,18.0,2020-01-21,2020-07-04,day,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",148151.0,858.0,7234.0,12.0,2020-01-21,2020-07-04,day,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",37661.0,0.0,1503.0,0.0,2020-01-21,2020-07-04,day,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",69724.0,302.0,3236.0,13.0,2020-01-21,2020-07-04,day,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


In [62]:
state_gdf.iloc[34]

NAME                                                  Virgin Islands
population                                                         0
fips                                                              78
dt_first_case                                             2020-03-14
dt_first_death                                            2020-04-06
cases_ts           0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
deaths_ts          0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
today_case                                                       111
today_new_case                                                     0
today_death                                                        6
today_new_death                                                    0
dt_start                                                  2020-01-21
dt_end                                                    2020-07-04
dt_unit                                                          day
geometry           (POLYGON ((-64.

In [63]:
state_case = state_gdf['today_case']
state_death = state_gdf['today_death']
state_case_per_100k_capita = state_gdf['today_case']/(state_gdf['population']/100000)
state_death_per_100k_capita = state_gdf['today_death']/(state_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [64]:
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.nan,0)
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.inf,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.nan,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.inf,0)

In [65]:
state_log_case = state_case.apply(lambda x: np.log(x + 1))
state_log_death = state_death.apply(lambda x: np.log(x + 1))
state_log_case_per_100k_capita = state_case_per_100k_capita.apply(lambda x: np.log(x + 1))
state_log_death_per_100k_capita =state_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [66]:
state_dict['case'] = {}
state_dict['death'] = {}
state_dict['case_per_100k_capita'] = {}
state_dict['death_per_100k_capita'] = {}

In [67]:
state_dict['case']['nolog'] = create_dict(state_case)
state_dict['case']['log'] = create_dict(state_log_case)
state_dict['death']['nolog'] = create_dict(state_death)
state_dict['death']['log'] = create_dict(state_log_death)
state_dict['case_per_100k_capita']['nolog'] = create_dict(state_case_per_100k_capita)
state_dict['case_per_100k_capita']['log'] = create_dict(state_log_case_per_100k_capita)
state_dict['death_per_100k_capita']['nolog'] = create_dict(state_death_per_100k_capita)
state_dict['death_per_100k_capita']['log'] = create_dict(state_log_death_per_100k_capita)

In [68]:
state_dict

{'case': {'nolog': {'Quantiles': {'bins': '3397.000000000001,13063.00000000001,24717.0,46717.00000000001,87592.00000000004,401286.0',
    'k': 6},
   'FisherJenks': {'bins': '24717.0,56183.0,109838.0,195707.0,258880.0,401286.0',
    'k': 6},
   'NaturalBreaks': {'bins': '21671.0,56183.0,109838.0,195707.0,258880.0,401286.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '8.130942302231878,9.4776156346503,10.11528700214968,10.751884808366633,11.380456365055815,12.902432161013119',
    'k': 6},
   'FisherJenks': {'bins': '4.718498871295094,7.382124365737512,8.960339366492091,10.11528700214968,11.45759244291205,12.902432161013119',
    'k': 6},
   'NaturalBreaks': {'bins': '4.718498871295094,7.382124365737512,8.960339366492091,10.11528700214968,11.45759244291205,12.902432161013119',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '94.0,286.00000000000034,721.0,1503.0000000000014,3278.000000000003,31860.0',
    'k': 6},
   'FisherJenks': {'bins': '721.0,1849.0,4335.0,8172.0,15189.

4. Read DPH Illinois County Data

In [69]:
dph_illinois_gdf = gpd.read_file('illinois/dph_county_data.geojson')

In [70]:
dph_illinois_gdf.head(5)

,id,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,dt_start,dt_end,dt_unit,population,geometry
0,McHenry,McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,2115,12,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,97,0,25822,409,2020-03-17,2020-07-05,day,308570,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
1,Boone,Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,608,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,21,0,5692,145,2020-03-17,2020-07-05,day,53577,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
2,Ogle,Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,271,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,4,0,4589,217,2020-03-17,2020-07-05,day,50923,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
3,Will,Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,6923,40,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,320,0,81099,949,2020-03-17,2020-07-05,day,692310,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
4,LaSalle,LaSalle,"0,0,1,1,1,1,1,3,3,3,3,3,4,5,5,6,6,6,7,8,11,12,...",2020-03-19,226,1,"0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,...",2020-03-29,17,0,8920,125,2020-03-17,2020-07-05,day,109430,"POLYGON ((-88.93885 41.62837, -88.93891 41.628..."


In [71]:
dph_illinois_gdf = dph_illinois_gdf[dph_illinois_gdf['id'] != 'Illinois']

In [72]:
illinois_case = dph_illinois_gdf['today_case']
illinois_death = dph_illinois_gdf['today_death']
illinois_case_per_100k_capita = dph_illinois_gdf['today_case']/(dph_illinois_gdf['population']/100000)
illinois_death_per_100k_capita = dph_illinois_gdf['today_death']/(dph_illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [73]:
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [74]:
illinois_log_case = illinois_case.apply(lambda x: np.log(x + 1))
illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [75]:
dph_illinois_dict['case'] = {}
dph_illinois_dict['death'] = {}
dph_illinois_dict['case_per_100k_capita'] = {}
dph_illinois_dict['death_per_100k_capita'] = {}

In [76]:
dph_illinois_dict['case']['nolog'] = create_dict(illinois_case)
dph_illinois_dict['case']['log'] = create_dict(illinois_log_case)
dph_illinois_dict['death']['nolog'] = create_dict(illinois_death)
dph_illinois_dict['death']['log'] = create_dict(illinois_log_death)
dph_illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
dph_illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

In [77]:
dph_illinois_dict

{'case': {'nolog': {'Quantiles': {'bins': '10.0,20.0,47.0,136.00000000000006,450.00000000000153,53531.0',
    'k': 6},
   'FisherJenks': {'bins': '608,3091,7856,9954,39001,53531', 'k': 6},
   'NaturalBreaks': {'bins': '608.0,3091.0,7856.0,9954.0,39001.0,53531.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '2.3978952727983707,3.044522437723423,3.871201010907891,4.919980925828125,6.111467339502681,10.888034884916266',
    'k': 6},
   'FisherJenks': {'bins': '1.6094379124341003,3.2188758248682006,4.382026634673881,5.8289456176102075,8.036573409707312,10.888034884916266',
    'k': 6},
   'NaturalBreaks': {'bins': '1.6094379124341003,3.2188758248682006,4.382026634673881,5.8289456176102075,8.036573409707312,10.888034884916266',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,1.0,5.0,19.00000000000003,2634.0',
    'k': 6},
   'FisherJenks': {'bins': '32,137,320,475,1993,2634', 'k': 6},
   'NaturalBreaks': {'bins': '32.0,137.0,320.0,475.0,1993.0,2634.0', 'k': 6}},
  'log': {'

6. Read DPH County Test (Static) and DPH Zipcode Case (Static)


In [78]:
dph_illinois_test_gdf = gpd.read_file('illinois/dph_county_static_data.geojson')
dph_zipcode_gdf = gpd.read_file('illinois/dph_zipcode_data.geojson')

In [79]:
dph_illinois_test_gdf = dph_illinois_test_gdf[dph_illinois_test_gdf['NAME'] != 'Illinois']

In [80]:
dph_zipcode_gdf

,id,confirmed_cases,total_tested,geometry
0,62359,0.0,0.0,"POLYGON ((-91.18290 40.02688, -91.17656 40.020..."
1,62376,0.0,0.0,"POLYGON ((-91.35721 40.17576, -91.35600 40.151..."
2,62312,0.0,0.0,"POLYGON ((-90.97797 39.78881, -90.97787 39.772..."
3,62365,0.0,0.0,"POLYGON ((-91.16584 39.83701, -91.14703 39.837..."
4,62246,10.0,704.0,"POLYGON ((-89.56254 38.91811, -89.56794 38.923..."
...,...,...,...,...
1379,60484,129.0,965.0,"POLYGON ((-87.70969 41.45644, -87.70949 41.456..."
1380,62948,28.0,1149.0,"MULTIPOLYGON (((-89.02159 37.74516, -89.03561 ..."
1381,62921,0.0,0.0,"POLYGON ((-89.09155 37.79258, -89.08896 37.807..."
1382,62915,0.0,0.0,"POLYGON ((-89.11994 37.78039, -89.12263 37.781..."


In [81]:
dph_illinois_tested = dph_illinois_test_gdf['total_tested']
dph_zipcode_tested = dph_zipcode_gdf['total_tested']
dph_zipcode_case = dph_zipcode_gdf['confirmed_cases']

In [82]:
dph_illinois_log_tested = dph_illinois_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_tested = dph_zipcode_tested.apply(lambda x: np.log(x + 1))
dph_zipcode_log_case = dph_zipcode_case.apply(lambda x: np.log(x + 1))

In [83]:
dph_illinois_dict['tested'] = {}
dph_illinois_dict['zipcode_case'] = {}
dph_illinois_dict['zipcode_tested'] = {}

In [84]:
dph_illinois_dict['tested']['nolog'] = create_dict(dph_illinois_tested)
dph_illinois_dict['tested']['log'] = create_dict(dph_illinois_log_tested)
dph_illinois_dict['zipcode_case']['nolog'] = create_dict(dph_zipcode_case)
dph_illinois_dict['zipcode_case']['log'] = create_dict(dph_zipcode_log_case)
dph_illinois_dict['zipcode_tested']['nolog'] = create_dict(dph_zipcode_tested)
dph_illinois_dict['zipcode_tested']['log'] = create_dict(dph_zipcode_log_tested)

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

In [95]:
dph_illinois_dict

{'case': {'nolog': {'Quantiles': {'bins': '10.0,20.0,47.0,136.00000000000006,450.00000000000153,53531.0',
    'k': 6},
   'FisherJenks': {'bins': '608,3091,7856,9954,39001,53531', 'k': 6},
   'NaturalBreaks': {'bins': '608.0,3091.0,7856.0,9954.0,39001.0,53531.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '2.3978952727983707,3.044522437723423,3.871201010907891,4.919980925828125,6.111467339502681,10.888034884916266',
    'k': 6},
   'FisherJenks': {'bins': '1.6094379124341003,3.2188758248682006,4.382026634673881,5.8289456176102075,8.036573409707312,10.888034884916266',
    'k': 6},
   'NaturalBreaks': {'bins': '1.6094379124341003,3.2188758248682006,4.382026634673881,5.8289456176102075,8.036573409707312,10.888034884916266',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,1.0,5.0,19.00000000000003,2634.0',
    'k': 6},
   'FisherJenks': {'bins': '32,137,320,475,1993,2634', 'k': 6},
   'NaturalBreaks': {'bins': '32.0,137.0,320.0,475.0,1993.0,2634.0', 'k': 6}},
  'log': {'

7. Read Worldwide Data

In [85]:
who_world_gdf = gpd.read_file('worldwide/who_world_data.geojson')

In [86]:
who_world_case = who_world_gdf['today_case']
who_world_death = who_world_gdf['today_death']

In [87]:
who_world_log_case = who_world_case.apply(lambda x: np.log(x + 1))
who_world_log_death = who_world_death.apply(lambda x: np.log(x + 1))

In [88]:
who_world_dict['case'] = {}
who_world_dict['death'] = {}

In [89]:
who_world_dict['case']['nolog'] = create_dict(who_world_case)
who_world_dict['case']['log'] = create_dict(who_world_log_case)
who_world_dict['death']['nolog'] = create_dict(who_world_death)
who_world_dict['death']['log'] = create_dict(who_world_log_death)

In [96]:
who_world_dict

{'case': {'nolog': {'Quantiles': {'bins': '17.500000000000007,278.33333333333337,1336.0,6282.000000000001,36026.50000000001,2776366.0',
    'k': 6},
   'FisherJenks': {'bins': '28857.0,109505.0,295599.0,681251.0,1539081.0,2776366.0',
    'k': 6},
   'NaturalBreaks': {'bins': '28857.0,109505.0,295599.0,681251.0,1539081.0,2776366.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '2.915797269678664,5.632370068128213,7.196206041496584,8.745534726216677,10.49203590801157,14.836653796481524',
    'k': 6},
   'FisherJenks': {'bins': '1.3862943611198906,4.718498871295094,7.133295954896068,9.234545060673,11.35401179341874,14.836653796481524',
    'k': 6},
   'NaturalBreaks': {'bins': '1.3862943611198906,4.718498871295094,7.133295954896068,9.234545060673,11.35401179341874,14.836653796481524',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,7.0,27.5,111.66666666666669,799.0000000000036,129226.0',
    'k': 6},
   'FisherJenks': {'bins': '3280.0,11408.0,29843.0,44198.0,63174.0,129226

8. Read Vulnerability Data

In [90]:
vul_gdf = gpd.read_file('illinois/vulnerability.geojson')

In [91]:
vul = vul_gdf['today_vul']

In [92]:
vul_dict['case'] = {}

In [93]:
vul_dict['case']['nolog'] = create_dict(vul)

In [97]:
vul_dict

{'case': {'nolog': {'Quantiles': {'bins': '0.299889051252253,0.35957040861011014,0.411668422803416,0.4772095669606803,0.5572160404671521,0.727610415817312',
    'k': 6},
   'FisherJenks': {'bins': '0.295716969483447,0.365844447962564,0.433357080061713,0.504123445666279,0.574320048173377,0.727610415817312',
    'k': 6},
   'NaturalBreaks': {'bins': '0.292150336769466,0.360876862084978,0.427683393745169,0.500060806281009,0.572213665500091,0.727610415817312',
    'k': 6}}}}

Summary and ouput


In [94]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
total_dict['dph_illinois'] = dph_illinois_dict
total_dict['who_world'] = who_world_dict
total_dict['vulnerability'] = vul_dict
# dynamic_dict['county'] = dynamic_county_dict
# dynamic_dict['state'] = dynamic_state_dict
# dynamic_dict['illinois'] = dynamic_illinois_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
# with open('dynamic_classes.json','w') as json_file:
#     json.dump(dynamic_dict, json_file)
print('done')

done
